In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
_n_circles_       = 25
_circle_geoms_    = []
_min_circle_sep_  = 20
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 2
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    cx, cy, r = random.randint(30, 570), random.randint(30, 370), random.randint(20,40)
    while circleOverlaps(cx,cy,r):
        cx, cy, r = random.randint(30, 570), random.randint(30, 370), random.randint(20,40)
    return cx, cy, r

for i in range(_n_circles_):
    cx, cy, r = findOpening()
    _circle_geoms_.append((cx,cy,r))

# points to connect
_pts_ = []
c0    = random.randint(0, len(_circle_geoms_)-1)
c1    =  random.randint(0, len(_circle_geoms_)-1)
while c1 == c0:
    c1 =  random.randint(0, len(_circle_geoms_)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c0]
_pts_.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
_pts_.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = _circle_geoms_[c1]
_pts_.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
_pts_.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

_pts2_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts2_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts2_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

_pts3_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts3_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts3_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

_pts4_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts4_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts4_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

_pts5_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts5_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts5_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

svg = '<svg width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    _dist_, _inter_  = rt.segmentIntersectsCircle((_pts_[1],_pts_[2]),_geom_)
    if _dist_ <= _geom_[2]:
        _color_ = '#ff0000'
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'

_path_ = f'M {_pts_[0][0]} {_pts_[0][1]}'
for i in range(1, len(_pts_)):
    _path_ += f' L {_pts_[i][0]} {_pts_[i][1]}'
svg += f'<path d="{_path_}" fill="none" stroke="#ff0000" stroke-width="0.2" />'

svg += f'<circle cx="{_pts_[0][0]}"  cy="{_pts_[0][1]}"  r="3" stroke="#000000" fill="#ff0000" />'
svg += f'<circle cx="{_pts_[-1][0]}" cy="{_pts_[-1][1]}" r="3" stroke="#000000" fill="#ff0000" />'

svg += '</svg>'
rt.displaySVG(svg)

In [ ]:
def calculatePathAroundCircles(pts, circle_geoms, radius_inc_test):
    def breakSegment(_segment_):
        if rt.segmentLength(_segment_) < 2.0:
            return _segment_
        for _geom_ in circle_geoms:
            _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
            _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
            if _dist_ <= _circle_plus_[2]:
                if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                    dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                    l        = sqrt(dx*dx+dy*dy)
                    dx,  dy  = dx/l, dy/l
                    pdx, pdy = -dy, dx 
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+_half_sep_), _geom_[1] + pdy*(_geom_[2]+_half_sep_)), (_segment_[1][0], _segment_[1][1])]
                else:
                    dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                    l      = sqrt(dx*dx+dy*dy)
                    dx, dy = dx/l, dy/l
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + dx*(_geom_[2]+_half_sep_), _geom_[1] + dy*(_geom_[2]+_half_sep_)), (_segment_[1][0], _segment_[1][1])]
        return _segment_

    last_length = 0
    _segments_  = []
    for _pt_ in pts:
        _segments_.append(_pt_)
    while last_length != len(_segments_):
        last_length    = len(_segments_)
        _new_segments_ = []
        for i in range(len(_segments_)-1):
            _new_ = breakSegment([_segments_[i], _segments_[i+1]])
            if len(_new_) == 3:
                _new_segments_.append(_new_[0])
                _new_segments_.append(_new_[1])
            else:
                _new_segments_.append(_new_[0])
        _new_segments_.append(_new_[-1])
        _segments_ = _new_segments_
        
    return _segments_

def createLinePathFromSegments(segments):
    _path_ = f'M {segments[0][0]} {segments[0][1]}'
    for i in range(1,len(segments)):
        _path_ += f' L {segments[i][0]} {segments[i][1]}'
    return _path_

def createCurvedPathFromSegments(segments, amp = 10.0):
    _path_ =  f'M {segments[0][0]} {segments[0][1]}'
    _path_ += f' L {segments[1][0]} {segments[1][1]}'
    for i in range(1,len(segments)-2):
        v0 = rt.unitVector([segments[i],   segments[i-1]])
        v1 = rt.unitVector([segments[i+1], segments[i+2]])
        _path_ += f' C {segments[i][0]-amp*v0[0]} {segments[i][1]-amp*v0[1]} {segments[i+1][0]-amp*v1[0]} {segments[i+1][1]-amp*v1[1]} {segments[i+1][0]} {segments[i+1][1]}'
    _path_ += f' L {segments[-1][0]} {segments[-1][1]}'
    return _path_


In [ ]:
my_segments  = calculatePathAroundCircles(_pts_,  _circle_geoms_, _radius_inc_test_)
my_segments2 = calculatePathAroundCircles(_pts2_, _circle_geoms_, _radius_inc_test_)
my_segments3 = calculatePathAroundCircles(_pts3_, _circle_geoms_, _radius_inc_test_)
my_segments4 = calculatePathAroundCircles(_pts4_, _circle_geoms_, _radius_inc_test_)
my_segments5 = calculatePathAroundCircles(_pts5_, _circle_geoms_, _radius_inc_test_)

svg2 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg2 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg2 += f'<path d="{createLinePathFromSegments(my_segments)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg2 += f'<circle cx="{_pts_[0][0]}"  cy="{_pts_[0][1]}"  r="3" stroke="#000000" fill="#ff0000" />'
svg2 += f'<circle cx="{_pts_[-1][0]}" cy="{_pts_[-1][1]}" r="3" stroke="#000000" fill="#ff0000" />'
svg2 += '</svg>'

rt.tile([svg,svg2])

In [ ]:
svg3 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg3 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg3 += f'<path d="{createCurvedPathFromSegments(my_segments)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg3 += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="1" stroke="#000000" fill="#ff0000" />'
svg3 += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg3 += '</svg>'
rt.tile([svg2,svg3])

In [ ]:
svg4l = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg4l += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments2)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments3)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments4)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments5)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += '</svg>'

svg4 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg4 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments2)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments3)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments4)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments5)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += '</svg>'
rt.tile([svg4l,svg4])